## EDA

In [401]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.models import Model

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import initializers
from keras.layers import regularizers
from keras.callbacks import EarlyStopping

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
 

from keras.optimizers import Adam, Adagrad, Adadelta

import keras.backend as K
from keras.utils.np_utils import to_categorical

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import chi2, SelectKBest

import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline


In [2]:
from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
set_random_seed(234)

pandas has pivot tables - R probably does too


In [40]:
df = pd.read_csv("../Processed data/6_4.csv")

In [41]:
df.head(1)
#df.columns

,Unnamed: 0,Segment.Pd,Account.Manager.Pd,Underwriter.Pd,Master.Agent.Cd.Pd,Prime.Agent.Pd,Prime.Agent.Cd.Pd,Sales.Director.Pd,Underwriting.Director.Pd,Brick...Mortar.Location.Pd,...,Developed.Exposure,Developed.New.Business,Developed.Other,Developed.Rate,Developed.Retention,Developed.RPC,Developed.Written.Premium,Rate.Change,Total.Insured.Value,retention_percent
0,1,Store,"Croghan,Luz","Denney,Melanie",0NP553,MITCHELL REED &SCHMITTEN,0NP553,"Quayle,Mel","Vinge,Alan",MITCHELL REED & SCHMITTEN INSURANCE INC,...,0.0,0,0.0,0.0,0.0,0.0,0.0,192.28,low,0.0


In [42]:
#df = df.drop(['Unnamed: 0'], axis = 1)

In [8]:
df.columns

Index(['Segment.Pd', 'Account.Manager.Pd', 'Underwriter.Pd',
       'Master.Agent.Cd.Pd', 'Prime.Agent.Pd', 'Prime.Agent.Cd.Pd',
       'Sales.Director.Pd', 'Underwriting.Director.Pd',
       'Brick...Mortar.Location.Pd', 'Sales.Executive.Pd', 'Pd.State.Cd',
       'Hybrid.Territory', 'Hybrid.County', 'Hybrid.Zip',
       'Construction.Type.Pd', 'Program.Pd', 'Program.Cd.Pd',
       'Plan.Location.Pd', 'Master.Agent.Pd', 'Line', 'MonoMulti1',
       'New.Renew.Ind', 'Quintile', 'Growth.Category', 'Industry.Grouping',
       'Line.Tenure', 'Developed.Base', 'Developed.Exposure',
       'Developed.New.Business', 'Developed.Other', 'Developed.Rate',
       'Developed.Retention', 'Developed.RPC', 'Developed.Written.Premium',
       'Rate.Change', 'Total.Insured.Value', 'retention_percent'],
      dtype='object')

In [81]:
junk = df[['Segment.Pd', 'Account.Manager.Pd', 'Underwriter.Pd', 'Master.Agent.Cd.Pd', 'Prime.Agent.Cd.Pd',
          'Sales.Director.Pd', 'Underwriting.Director.Pd', 'Brick...Mortar.Location.Pd', 'Pd.State.Cd',
          'Hybrid.Territory', 'Hybrid.Zip']]

### Some manual cleaning

In [85]:
junk.shape

(200, 11)

In [83]:
junk.head(2) #master and agent are not always equal

,Segment.Pd,Account.Manager.Pd,Underwriter.Pd,Master.Agent.Cd.Pd,Prime.Agent.Cd.Pd,Sales.Director.Pd,Underwriting.Director.Pd,Brick...Mortar.Location.Pd,Pd.State.Cd,Hybrid.Territory,Hybrid.Zip
0,Store,"Croghan,Luz","Denney,Melanie",0NP553,0NP553,"Quayle,Mel","Vinge,Alan",MITCHELL REED & SCHMITTEN INSURANCE INC,western,3,98826
1,Office,"Rossi,Judy","Rybeck,Chris",0CCJ01,0CCJ01,"Quayle,Mel","Vinge,Alan",DML INSURANCE SERVICES,western,6,98021


In [84]:
junk = junk[0:200]

y = df['New.Renew.Ind']

y = y[0:200, ]

y = pd.get_dummies(y, drop_first = True)

y.head(2)

In [91]:
#test['combo'] = test['Account.Manager.Pd'] + " " + test['Segment.Pd']

In [92]:
junk.iloc[:2, :4]

,Segment.Pd,Account.Manager.Pd,Underwriter.Pd,Master.Agent.Cd.Pd
0,Store,"Croghan,Luz","Denney,Melanie",0NP553
1,Office,"Rossi,Judy","Rybeck,Chris",0CCJ01


In [93]:
junk = junk.astype(str)

In [95]:
#junk = test.iloc[:,0:4].apply(lambda x: ' '.join(x), axis=1)

In [96]:
junk = junk.apply(lambda x: ' '.join(x), axis=1) 

In [97]:
print(junk.head(1))
junk.shape #should be a column vec

0    Store Croghan,Luz Denney,Melanie 0NP553 0NP553...
dtype: object


(200,)

In [98]:
f = lambda name: name.replace(",", " ").lower()


In [99]:
junk = junk.map(f)

In [128]:
junk[0:3]

0    store croghan luz denney melanie 0np553 0np553...
1    office rossi judy rybeck chris 0ccj01 0ccj01 q...
2    office mccomb jill fuss amanda 0g0355 0g3400 b...
dtype: object

In [129]:
# find the max line length
count = []
for sent in range(len(junk)):
    sent = len(junk[sent].split())
    count.append(sent)
      
max(count)    
    

21

In [130]:
token = Tokenizer(num_words = 21)

In [131]:
#token.fit_on_texts(['Store Croghan Luz Denney Melanie 0NP553', 'Office Rossi Judy Rybeck Chris 0CCJ01' ])
#print(token.word_index)

In [132]:
token.fit_on_texts(junk)

In [133]:
type(token.word_index)

dict

In [136]:
dict = token.word_index
max(dict.values())

785

In [ ]:
rev_dict = {}

In [71]:
#print(token.word_counts)

In [117]:
#X = token.texts_to_matrix(junk) #pay dirt

In [374]:
df.columns

Index(['Segment.Pd', 'Account.Manager.Pd', 'Underwriter.Pd',
       'Master.Agent.Cd.Pd', 'Prime.Agent.Pd', 'Prime.Agent.Cd.Pd',
       'Sales.Director.Pd', 'Underwriting.Director.Pd',
       'Brick...Mortar.Location.Pd', 'Sales.Executive.Pd', 'Pd.State.Cd',
       'Hybrid.Territory', 'Hybrid.County', 'Hybrid.Zip',
       'Construction.Type.Pd', 'Program.Pd', 'Program.Cd.Pd',
       'Plan.Location.Pd', 'Master.Agent.Pd', 'Line', 'MonoMulti1',
       'New.Renew.Ind', 'Quintile', 'Growth.Category', 'Industry.Grouping',
       'Line.Tenure', 'Developed.Base', 'Developed.Exposure',
       'Developed.New.Business', 'Developed.Other', 'Developed.Rate',
       'Developed.Retention', 'Developed.RPC', 'Developed.Written.Premium',
       'Rate.Change', 'Total.Insured.Value', 'retention_percent'],
      dtype='object')

### Keras cleaning

In [209]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM

In [375]:
junk = df[['Segment.Pd', 'Account.Manager.Pd', 'Underwriter.Pd', 'Master.Agent.Cd.Pd', 'Prime.Agent.Cd.Pd',
          'Sales.Director.Pd', 'Underwriting.Director.Pd', 'Brick...Mortar.Location.Pd', 'Pd.State.Cd',
          'Hybrid.Territory', 'Hybrid.Zip', 'Construction.Type.Pd', 'Program.Pd', 'Plan.Location.Pd',
          'Line', 'MonoMulti1','New.Renew.Ind', 'Quintile', 'Growth.Category', 'Industry.Grouping']]

In [376]:
y = df['New.Renew.Ind']

#y = y[0:200, ]

y = pd.get_dummies(y, drop_first = True)

labels = np.asarray(y)

type(labels)

numpy.ndarray

In [377]:
np.unique(y, return_counts = True)

(array([0, 1], dtype=uint8), array([ 554, 4959], dtype=int64))

In [378]:
junk = junk.astype(str)
junk = junk.apply(lambda x: ' '.join(x), axis=1) 

#junk = junk[0:200]

In [379]:
junk.shape

(5513,)

In [380]:
tokenizer = Tokenizer(num_words = None, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', 
                      lower=True, split=' ') #this is default btw

In [381]:
tokenizer.fit_on_texts(junk)

In [382]:
tokenizer

In [383]:
sequences = tokenizer.texts_to_sequences(junk) # list

In [384]:
# compare, just note that the cleaned data is now a keras object, Junk is not cleaned
print(sequences[0:1])
print(junk[0:1])

[[44, 74, 75, 115, 116, 680, 680, 23, 24, 18, 19, 1115, 1116, 1244, 10, 21, 2, 25, 1995, 34, 44, 7, 470, 17, 8, 3, 9, 3, 54, 1, 13, 4, 35, 70]]
0    Store Croghan,Luz Denney,Melanie 0NP553 0NP553...
dtype: object


In [385]:
word_index = tokenizer.word_index # type = dict

In [266]:
#word_index

In [386]:
print('Found {} unique tokens.'.format(len(word_index)))

Found 4560 unique tokens.


In [387]:
data = pad_sequences(sequences, maxlen = None) #numpyarray of dim (samples, maxlen)

In [388]:
len(data[0])
len(data[1]) #all elements are padded to the max length

41

In [389]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5513, 41)
Shape of label tensor: (5513, 1)


In [390]:
rev_word_index = {v+3:k for k,v in word_index.items()}

In [391]:
#rev_word_index

In [392]:
rev_word_index[0] = 'padding_char'
rev_word_index[1] = 'start_char'
rev_word_index[2] = 'oov_char'
rev_word_index[3] = 'unk_char'

In [393]:
data[0:2, ]

array([[   0,    0,    0,    0,    0,    0,    0,   44,   74,   75,  115,
         116,  680,  680,   23,   24,   18,   19, 1115, 1116, 1244,   10,
          21,    2,   25, 1995,   34,   44,    7,  470,   17,    8,    3,
           9,    3,   54,    1,   13,    4,   35,   70],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   27,
         162,  166,  169,  170,  766,  766,   23,   24,   18,   19, 1393,
          10,   20,    2,   68, 2819,   34,  163,  183,   17,    8,    3,
           9,    5,    1,   25,    4,   15,   69,   20]])

In [394]:
example_feat = ' '.join([rev_word_index[word] for word in data[3]])
example_feat #notice the zeros -> this is the padding. All lowercase, etc -> cleaned


'padding_char padding_char padding_char padding_char padding_char padding_char padding_char padding_char padding_char mccomb service contractors debbie bogdanski 0xy762 0xy762 services inc superior denver pac oov_char insurance 0cdn99 masonry amanda 0 80632 091773 0csp77 property mono unk_char nan unk_char start_char 2 renew kyle sarah mccomb'

In [395]:
X_train, X_test, y_train, y_test = train_test_split(data, labels,
                                                    test_size=0.3)


In [396]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)
# I don't see an issue with splitting after preprocessing, but I could be wrong

(3859, 41)
(1654, 41)
(1654, 1)
(3859, 1)


In [397]:
max_features = max([max(x) for x in X_train] + 
                   [max(x) for x in X_test]) + 1
max_features # number of features we have in the dictionary


4561

In [407]:
K.clear_session()

model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [409]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          callbacks=[EarlyStopping(monitor='acc', patience=1)],
          validation_split=0.3)


Train on 2701 samples, validate on 1158 samples
Epoch 1/10
2701/2701 [==============================] - 7s 3ms/step - loss: 0.3582 - acc: 0.8934 - val_loss: 0.2015 - val_acc: 0.9111
Epoch 2/10
2701/2701 [==============================] - 6s 2ms/step - loss: 0.0638 - acc: 0.9759 - val_loss: 0.0073 - val_acc: 1.0000
Epoch 3/10
2701/2701 [==============================] - 6s 2ms/step - loss: 0.0068 - acc: 0.9996 - val_loss: 0.0023 - val_acc: 1.0000
Epoch 4/10
2701/2701 [==============================] - 6s 2ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 9.1633e-04 - val_acc: 1.0000
Epoch 5/10
2701/2701 [==============================] - 6s 2ms/step - loss: 9.1434e-04 - acc: 1.0000 - val_loss: 4.8797e-04 - val_acc: 1.0000


In [410]:
score, acc = model.evaluate(X_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)


1654/1654 [==============================] - 1s 431us/step
Test score: 0.00382604211435
Test accuracy: 0.998790810157


In [411]:
y_pred = model.predict(X_test)


In [412]:
y_pred_class = [ ]
for i in y_pred:
    if i >= 0.5:
        y_pred_class.append(1)
    else:
        y_pred_class.append(0)


In [413]:
confusion_matrix(y_test, y_pred_class) # so this predicted every value as a renewal..

array([[ 174,    0],
       [   2, 1478]], dtype=int64)

In [414]:
print(classification_report(y_test, y_pred_class))
#very misleading -> try balancing the data

             precision    recall  f1-score   support

          0       0.99      1.00      0.99       174
          1       1.00      1.00      1.00      1480

avg / total       1.00      1.00      1.00      1654



Notes: hmmm. This is super accurate. However, I can't say that it is overfitting. The confusion matrix is pretty clear.  

test with titance - maybe you can improve your score  
Send code to Bagui and Bo  
Consider taking another shot at the hit ratio  
Consider combining with CNN